In [19]:
import requests
import bs4
from bs4 import BeautifulSoup
from time import sleep

import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

In [ ]:
indeed_cities = ['New+York', 'Chicago', 'San+Francisco', 'San+Jose', 'San+Diego', 'Los+Angeles', 'Washington%2C+DC',
                 'Boston', 'Pittsburgh', 'Philadelphia', 'Atlanta', 'Cincinnati', 'St.+Louis', 'Tampa', 'Oakland',
                 'Austin', 'Houston', 'Dallas', 'Seattle', 'Portland', 'Denver', 'Phoenix', 'Minneapolis', 'Miami',
                 'Charlotte', 'Jacksonville', 'Indianapolis', 'Nashville', 'Kansas+City', 'Columbus']
len(indeed_cities)

URL = 'https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=10'

In [ ]:
appended_results = []

def parser(URL):
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, 'lxml')
    
    #results_df = pd.DataFrame(columns = ['Job_title', 'Company', 'Location', 'Salary', 'Job_description'])
    
    for result in soup.find_all('div', {'class': 'row'}):
        try:
            title = result.find('h2', {'class' : 'title'}).text.replace('\n', '')
        except:
            title = 'NA'
   
        try:
            location = result.find('div', {'class' : 'recJobLoc'})["data-rc-loc"]
        except:
            location = 'NA'
        
        try:
            company = result.find('span', {'class' : 'company'}).text.replace('\n', '')
        except:
            company = 'NA'
                
        try:
            salary = result.find('span', {'class' : 'no-wrap'}).text
        except:
            salary = 'NA'
        
        job_description = result.find('div', {'class' : 'summary'}).text.replace('\n', '')
        
        data_parsed = {
                            'Job_title' : title, 'Company' : company,
                            'Location' : location, 'Salary' : salary,
                            'Job_description' : job_description
                          }
        appended_results.append(data_parsed)
    
    results_df = pd.DataFrame(appended_results)
    return results_df

In [ ]:
parser(URL)

In [ ]:
url_template = 'https://www.indeed.com/jobs?q=data+analyst&l={}&start={}'
max_results_per_city = 1000

city_list = [
             'New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta',
             'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Charlottesville',
             'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose' 'Austin', 'Jacksonville',
             'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 'Memphis', 'Boston', 'Nashville',
             'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 'Fresno', 'Sacramento', 'Long+Beach', 'Mesa',
             'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs', 'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis',
             'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora',
             'Santa+Ana', 'Riverside', 'Corpus+Christi', 'Pittsburgh', 'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge',
             'Chesapeake', 'Alexandria', 'Fairfax', 'Herndon', 'Reston', 'Roanoke'
            ]

for city in city_list:
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city, start)
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'lxml')
        
        for result in soup.find_all('div', {'class': 'row'}):
            try:
                title = result.find('h2', {'class' : 'title'}).text.replace('\n', '')
            except:
                title = 'NA'

            try:
                location = result.find('div', {'class' : 'recJobLoc'})["data-rc-loc"]
            except:
                location = 'NA'

            try:
                company = result.find('span', {'class' : 'company'}).text.replace('\n', '')
            except:
                company = 'NA'

            try:
                salary = result.find('span', {'class' : 'no-wrap'}).text
            except:
                salary = 'NA'

            job_description = result.find('div', {'class' : 'summary'}).text.replace('\n', '')

            data_parsed = {
                                'Job_title' : title, 'Company' : company,
                                'Location' : location, 'Salary' : salary,
                                'Job_description' : job_description
                              }
            appended_results.append(data_parsed)
    
        results_df = pd.DataFrame(appended_results)
print(results_df)
        
        


In [ ]:
results_df.shape

In [ ]:
results_df.to_csv('Indeed_uncleaned_data.csv', encoding = 'utf-8')

In [ ]:
X = pd.read_csv('Indeed_uncleaned_data.csv')

In [ ]:
X.drop('Unnamed: 0', axis = 1, inplace = True)


In [ ]:
print(X.head())
X.shape

In [ ]:
# ---------------------- DATA PREPROCESSING -----------------------------------

X = X.drop_duplicates()
X.reset_index(drop = True, inplace = True)
X.shape

In [ ]:
X['Salary'].value_counts()

In [ ]:
# removing unwanted charcters from 'Salary' column

X['Salary'] = X['Salary'].str.replace("\n", '')
X['Salary'] = X['Salary'].str.replace("$", '')
X['Salary'] = X['Salary'].str.replace(",", '')
X['Salary'] = X['Salary'].str.replace("a year", '')
X['Salary'] = X['Salary'].str.replace("From", '')

In [ ]:
X.head()
print(X.shape)

In [ ]:
X = X[X.Salary.str.contains("hour") == False]
X = X[X.Salary.str.contains("month") == False]
X = X[X.Salary.str.contains("week") == False]
X = X[X.Salary.str.contains("Up") == False]
X = X[X.Salary.str.contains("class") == False]
X = X[X.Salary.str.contains("day") == False]

print(X.shape)
X.head()

In [ ]:
#X['Salary'].unique()

In [ ]:
new_salary_list = []
for i in X['Salary']:
    a = i.split('-')
    if len(a) == 2:
        new_salary_list.append(np.mean([float(b) for b in a]))
    else:
        new_salary_list.append(float(a[0]))

In [ ]:
new_salary_list[0:5]

In [ ]:
X['Salary'] = new_salary_list

In [ ]:
X.shape

In [ ]:
X.duplicated().sum()

In [ ]:
X.head()

In [ ]:
X['Location'].isnull().sum()

In [ ]:
X.to_csv('Indeed_cleaned_data.csv', index = False, encoding = 'utf-8')

In [3]:
df = pd.read_csv('Indeed_cleaned_data.csv')

In [4]:
df.head()

,Job_title,Company,Location,Salary,Job_description
0,Audit Data Analyst,Bellwether Staffing Solutions,"White Plains, NY",87500.0,We are currently recruiting for an Audit Data ...
1,Jr. Business Analyst,First Notch Technology,"Nyack, NY",55000.0,As a Business Analyst based in Halifax you are...
2,Business Analyst,Hixny,"Albany, NY",57500.0,This detail and process-oriented position will...
3,Senior Data Analyst,Labaton Sucharow,"New York, NY",80000.0,3-6 years of related experience;The candidate ...
4,Entry Level Business Analyst,Digi555,"New York, NY",64000.0,We are looking to hire Entry Level Business An...


In [5]:
median_salary = np.median(df.Salary)
print(median_salary)

77500.0


In [6]:
df['_high_Salary'] = [1 if i > median_salary else 0 for i in df.Salary]
df.head()

,Job_title,Company,Location,Salary,Job_description,_high_Salary
0,Audit Data Analyst,Bellwether Staffing Solutions,"White Plains, NY",87500.0,We are currently recruiting for an Audit Data ...,1
1,Jr. Business Analyst,First Notch Technology,"Nyack, NY",55000.0,As a Business Analyst based in Halifax you are...,0
2,Business Analyst,Hixny,"Albany, NY",57500.0,This detail and process-oriented position will...,0
3,Senior Data Analyst,Labaton Sucharow,"New York, NY",80000.0,3-6 years of related experience;The candidate ...,1
4,Entry Level Business Analyst,Digi555,"New York, NY",64000.0,We are looking to hire Entry Level Business An...,0


In [7]:
df.Location.value_counts()

Washington, DC     94
New York, NY       38
Boston, MA         25
Arlington, VA      24
Atlanta, GA        21
                   ..
Braintree, MA       1
Orange, CA          1
Bangor, WA          1
Endicott, NY        1
New Orleans, LA     1
Name: Location, Length: 183, dtype: int64

In [8]:
city_name = []
state_name = []
for loc in df.Location:
    items = loc.split(',')
    city_name.append(items[0])
    state_name.append(items[1])

In [9]:
import re
only_states = []

for state in state_name:
    only_states.append(re.search(r'\w+', state).group(0))

In [10]:
df['City Name'] = city_name
df['State'] = only_states

In [11]:
df.head()

,Job_title,Company,Location,Salary,Job_description,_high_Salary,City Name,State
0,Audit Data Analyst,Bellwether Staffing Solutions,"White Plains, NY",87500.0,We are currently recruiting for an Audit Data ...,1,White Plains,NY
1,Jr. Business Analyst,First Notch Technology,"Nyack, NY",55000.0,As a Business Analyst based in Halifax you are...,0,Nyack,NY
2,Business Analyst,Hixny,"Albany, NY",57500.0,This detail and process-oriented position will...,0,Albany,NY
3,Senior Data Analyst,Labaton Sucharow,"New York, NY",80000.0,3-6 years of related experience;The candidate ...,1,New York,NY
4,Entry Level Business Analyst,Digi555,"New York, NY",64000.0,We are looking to hire Entry Level Business An...,0,New York,NY


In [12]:
df['City Name'].nunique()
print(df['City Name'].value_counts())

Washington      94
New York        38
Boston          25
Arlington       24
Atlanta         21
                ..
Randolph AFB     1
Rome             1
Leander          1
Fort Meade       1
Catonsville      1
Name: City Name, Length: 182, dtype: int64


In [13]:
city_dummy = pd.get_dummies(df['City Name'])
X_city = city_dummy
Y_city = df['_high_Salary']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_city, Y_city, test_size = 0.3, random_state = 80)

In [15]:
rf_model = RandomForestClassifier(n_estimators = 300, random_state = 90)
rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
acc = accuracy_score(y_test, rf_predictions)
print("Accuracy Score: ", acc.round(3))

s = cross_val_score(rf_model, X_city, Y_city, cv = 10, n_jobs = -1)
print('Cross Validation Score:\t{:0.3} ± {:0.3}'.format(s.mean().round(3), s.std().round(3)))

Accuracy Score:  0.663
Cross Validation Score:	0.516 ± 0.13


In [18]:
feature_importance = pd.DataFrame(rf_model.feature_importances_, index = X_city.columns).reset_index()
feature_importance.columns = ['Feature', 'importance']
feature_importance_median = []

for i in X_city.columns:
    feature_importance_median.append(np.median(df[df['City Name'] == i].Salary))

feature_importance['median_salary'] = feature_importance_median
feature_importance['Over or Under'] = [1 if i > median_salary else 0 for i in feature_importance.median_salary]
feature_importance.sort_values('importance', ascending = False).head(15)

,Feature,importance,median_salary,Over or Under
177,Washington,0.037812,94416.25,1
125,Phoenix,0.027173,62500.00,0
132,Queens,0.024158,81167.00,1
50,Dallas,0.024117,72500.00,0
150,San Francisco,0.022986,102500.00,1
19,Boston,0.022384,106100.00,1
8,Atlanta,0.020290,61000.00,0
11,Baltimore,0.017476,75000.00,0
76,Indianapolis,0.016302,102136.50,1
163,Tampa,0.015356,55000.00,0


In [22]:
df_desc = df[df.Job_description.notnull()]
X_desc = df_desc.Job_description
y_desc = df_desc._high_Salary

#df_desc.head()

In [23]:
cv = CountVectorizer(stop_words = 'english')
cv.fit(X_desc)

CountVectorizer(stop_words='english')

In [24]:
len(cv.get_feature_names())

2187

In [25]:
X_desc_transformed = pd.DataFrame(cv.transform(X_desc).todense(), columns = cv.get_feature_names())

In [26]:
X_desc_train, X_desc_test, y_desc_train, y_desc_test = train_test_split(np.asmatrix(X_desc_transformed), y_desc, test_size = 0.3, random_state = 60, stratify = y_desc)

In [28]:
word_counts = X_desc_transformed.sum(axis = 0)
word_counts.sort_values(ascending = False).head(20)

data           668
years          225
experience     197
business       178
analyst        167
analysis       131
management      82
position        78
analyze         67
required        67
support         57
research        57
skills          54
work            53
reports         53
program         51
information     48
attention       48
using           47
degree          46
dtype: int64

In [31]:
word_counts.to_csv('Indeed_wordCounts.csv', index = False, encoding = 'utf-8')

c:\users\amanj\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [34]:
rf_desc_model = RandomForestClassifier(200, random_state = 59)
rf_desc_model.fit(X_desc_train, y_desc_train)

rf_desc_model_preds = rf_desc_model.predict(X_desc_test)
desc_model_acc = accuracy_score(y_desc_test, rf_desc_model_preds)
print("Accuracy: ", desc_model_acc.round(3))

s = cross_val_score(rf_desc_model, X_desc_transformed.as_matrix(), y_desc.as_matrix(), cv = 10, n_jobs = -1)
print("Cross Validation Score: {0:3} ± {0:3}".format(s.mean().round(3), s.std().round(3)))

Accuracy:  0.639


c:\users\amanj\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Cross Validation Score: 0.618 ± 0.618


In [35]:
feature_importance = pd.DataFrame(rf_desc_model.feature_importances_, index = X_desc_transformed.columns).reset_index()
feature_importance.columns = ['Features', 'Importance']

FI_median = []
FI_mean = []

for i in X_desc_transformed.columns:
    FI_median.append(np.median(df_desc[df_desc.Job_description.str.lower().str.contains(i)].Salary))
    FI_mean.append(np.mean(df_desc[df_desc.Job_description.str.lower().str.contains(i)].Salary))

feature_importance['median_salary'] = FI_median
feature_importance['mean_salary'] = FI_mean
feature_importance['over_or_under'] = [1 if i > median_salary else 0 for i in feature_importance.median_salary]

feature_importance.sort_values('Importance', ascending = False).head(20)

,Features,Importance,median_salary,mean_salary,over_or_under
571,data,0.010150,75000.00,81488.768802,0
169,analyze,0.009648,71469.50,72862.184932,0
2182,years,0.008883,80193.25,85855.216049,1
803,experience,0.008857,80000.00,83634.570588,1
1587,program,0.008133,86000.00,86939.607692,1
605,department,0.007670,88339.50,89760.585714,1
1713,reports,0.007257,62500.00,71361.905660,0
90,accuracy,0.007200,70000.00,66887.761905,0
161,analysis,0.006363,80000.00,83438.088496,1
593,degree,0.006321,85000.00,92298.621622,1
